# Collaboration and Competition

My submission for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

This is the central file you want to use. It contains
-    all the import statements and instructions to start the environment
-    calls to train
-    calls to test

Description and justification of the model architectures and hyperparameters, plot of rewards and ideas for future works can be found in my [report](../report.ipynb).

For more information, I encourage you to have a look at my [description of the project](../README.md) as well.

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

---

In [ ]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
import time
import os
import pickle
from itertools import count
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="../Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents = 2
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action = 2
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space -- Each observes a state with length: 24
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

The **observation space** consists of **8 variables** corresponding to
- the `position [x, y]`
- and `velocity [vx, vy]`
- of the `ball`
- and `racket`.

Each agent receives its own, local observation. **Two continuous actions** are available, corresponding to

- `movement toward (or away from) the net`
- and `jumping`. 

For each agent, the **"state"** issued by the environment is actually the **concatenation of 3 successive observations**. The most recent being at the end.

- Hence, for each agent `len(state) = 24`

Let's define a function to print the actions and states for the two agents.

In [ ]:
# !pip install tabulate
# print transitions s-a-s'-a'-s'' for one single agent
from tabulate import tabulate
def print_state(single_state, actions_deque, agent_name):
    full_actions = []
    for i in range(3):
        full_actions = full_actions + [""]*2  # padding
        full_actions.append("{:.3f}".format(actions_deque[i][0]))
        full_actions.append("{:.3f}".format(actions_deque[i][1]))
        full_actions = full_actions + [""]*4  # padding
    state_vector_names = ['racket pos x', 'racket pos y', 'racket vel x', 'racket vel y',
                          'ball pos x', 'ball pos y', 'ball vel x', 'ball vel y']
    res = []
    for i, e in enumerate(state_vector_names):
        line = []
        for k in range(3):
            line.append(full_actions[i+k*8])
            line.append(single_state[i+k*8])        
        line[0] = e
        res.append(line)
    headers = [agent_name, "s_t-2", "a_t-2", "s_t-1", "a_t-1", "s_t"]
    print(tabulate(res, headers=headers, numalign="right", floatfmt=".3f"))

# print and compare states for both agents
def print_both_states(full_state):
    state_vector_names = ['racket pos x', 'racket pos y', 'racket vel x', 'racket vel y',
                          'ball pos x', 'ball pos y', 'ball vel x', 'ball vel y']
    res = []
    for i, e in enumerate(state_vector_names):
        line = [[full_state[0][i+k*8], full_state[1][i+k*8]] for k in range(3)]
        line = [e for l in line for e in l]
        line.insert(0, e)
        res.append(line)
    headers = ["", "st-2_A1", "st-2_A2", "st-1_A1", "st-1_A2", "st_A1", "st_A2"]
    print(tabulate(res, headers=headers, numalign="right", floatfmt=".3f"))

In [ ]:
print('The initial states for the both agents look like:')
print_state(states[0], [[-1, -1], [0, 0], [1, 1]], "Agent_0")

In [ ]:
# env.close()

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
random_actions = False

In [ ]:
# run steps in the environment. Either random or controlled by an agent
def show(n_runs, random_flag):
    actions_deque = deque(maxlen=3)  # collect 3 latest actions
    actions_deque.append([[0, 0], [0, 0]])
    actions_deque.append([[0, 0], [0, 0]])
    actions_deque.append([[0, 0], [0, 0]])
    for i in range(1, n_runs):
        env_info = env.reset(train_mode=False)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        print_both_states(states)
        print_state(states[0], [a[0] for a in actions_deque], "Agent_0")
        print_state(states[1], [a[1] for a in actions_deque], "Agent_1")
        while True:
            if random_flag:
                actions = np.random.randn(num_agents, action_size)
                actions = np.clip(actions, -1, 1)
            else:
                actions = agent.act((states), noise=0)
            actions_deque.append(actions)

            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            print_both_states(states)
            print_state(states[0], [a[0] for a in actions_deque], "Agent_0")
            print_state(states[1], [a[1] for a in actions_deque], "Agent_1")
            if np.any(dones):                                  # exit loop if episode finished
                print('\nDONE')
                break
        print('Score (max over agents) from episode {}: {}\n'.format(i, np.max(scores)))

In [ ]:
if random_actions:
    show(n_runs=6, random_flag=True)

When finished, you can close the environment.

In [ ]:
if random_actions:
    env.close()

### 4. Training

When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
model_dir= os.getcwd()+"/checkpoints"
os.makedirs(model_dir, exist_ok=True)

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]
scores_global = []

In [ ]:
def ddpg(name, n_max_episodes=3000):
    noise = 1.0
    mini_noise = 0.02
    noise_reduction = 0.9977

    scores_global = []                                    # collect scores for each episode
    scores_deque = deque(maxlen=100)                      # collect 100 latest scores
    start_learning = time.time()

    for i_episode in range(1, n_max_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment    
        states = env_info.vector_observations             # get the current state (for each agent)
        scores = np.zeros(num_agents)                     # initialize the score (for each agent)
        agent.reset()                                     # reset noise process
        noise *= noise_reduction

        start_episode = time.time()
        for _ in count():
            actions = agent.act((states), noise=max(noise, mini_noise))

            env_info = env.step(actions)[brain_name]      # send all actions to the environment
            next_states = env_info.vector_observations    # get next state (for each agent)
            rewards = env_info.rewards                    # get reward (for each agent)
            dones = env_info.local_done                   # see if episode finished
            scores += rewards                             # update the score (for each agent)

            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done)
            
            if np.any(dones):                             # exit loop if episode finished
                break
            
            states = next_states                          # roll over states to next time step

#         if sum(n > 0 for n in scores):
#             print(scores)

        score = np.max(scores)                           # average over 20 agents for this episode
        scores_deque.append(score)
        scores_global.append(score)
        
        if i_episode % 10 == 0:
            print('epi={} noise={:.2f} score[ep]={:.2f} mean[100]={:.2f} max[100]={:.2f} min[100]={:.2f} std[100]={:.2f} dur[ep]={:.2f}'\
                  .format(i_episode, noise, score, np.mean(scores_deque), np.max(scores_deque), np.min(scores_deque), np.std(scores_deque), time.time()-start_episode), end="\n")
        
        if i_episode % 50 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoints/actor_-'+str(name)+"_"+str(i_episode)+'.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoints/critic_-'+str(name)+"_"+str(i_episode)+'.pth')
        
        if np.mean(scores_deque) >= 0.5:
            print("SUCCESS -- after {} episodes and {:.2f} min".format(i_episode, (time.time()-start_learning)/60))
            torch.save(agent.actor_local.state_dict(), 'checkpoints/actor_'+str(name)+'success.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoints/critic_'+str(name)+'success.pth')
            return scores_global
            
    return scores_global

In [ ]:
# each choice of hyperparameters is executed under five random seeds
seeds = [random.randint(1, 100) for _ in range(5)]
print(seeds)
for i in seeds:
    print("--- seed = {} ---".format(i))
    agent = Agent(state_size=state_size, action_size=action_size, random_seed=i)
    scores = ddpg(name=i)

    print(scores)
    # save scores
    logfile_name = str(i)+".txt"
    with open(logfile_name, "wb") as fp:
        pickle.dump(scores, fp)

### 5. Testing

In [ ]:
# Load the saved model weights of a successful agent
agent = Agent(state_size=state_size, action_size=action_size, random_seed=67)
agent.actor_local.load_state_dict(torch.load('checkpoints/checkpoint_actor_x67success.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoints/checkpoint_critic_x67success.pth'))

In [ ]:
show(n_runs=12, random_flag=False)

In [ ]:
env.close()